In [ ]:
from importlib import reload
from pathlib import Path
from ipynb.fs.full.notebook_viewer import Viewer, Manager
root = Path("/data/natsuki/training116/00030-v4-mirror-auto4-gamma100-batch64-noaug-resumecustom/011289")
assert root.is_dir()
filelist = list(root.glob("*"))
filelist.sort()

In [ ]:

Viewer(
    filelist,
    N=23,
    key="nosort",
    root=root,
)